# Reddit 크롤러

- pushshift API 사용
- 출처: https://github.com/S00ahKim/Reddit-Submissions-Analysis

In [1]:
import requests
from datetime import datetime
import time
from dateutil.relativedelta import relativedelta
import datetime as dt
import pandas as pd
import urllib.request
import urllib.error
import os

In [2]:
def cal_time(tmstmp, hour):
    '''
    시간 계산기: 타임스탬프에 주어진 인자만큼의 시간을 더한 타임스탬프를 리턴
    '''
    tmp = int(tmstmp)
    dttime = datetime.fromtimestamp(tmp)
    wttime = dttime + dt.timedelta(hours = hour)
    return str(int(wttime.timestamp()))

In [3]:
def get_server_time():
    '''
    reddit 서버 시간 구하기
    '''
    date = urllib.request.urlopen('http://www.google.com').headers['Date']
    timestmp = int(time.mktime(time.strptime(date, '%a, %d %b %Y %H:%M:%S %Z')))
    return timestmp

In [4]:
def encode_roman(x):
    '''
    인코딩할 수 없는 문자열 제거
    '''
    return str(x).encode('ascii', 'ignore').decode('ascii')

In [5]:
def cal_end_by_start(start):
    '''
    시작 시간을 인자로 주면 한달 뒤의 시간을 문자열로 리턴한다.
    '''
    tmp = int(start)
    dttime = datetime.fromtimestamp(tmp)
    wttime = dttime + relativedelta(months=1)
    return str(int(wttime.timestamp()))

In [6]:
def get_start_time(subreddit):
    meta = pd.read_csv('./data/metadata/data.csv', header=0)
    try:
        start = meta.loc[meta['subreddit'] == subreddit, 'last'].values[0]
    except:
        r = requests.get('https://www.reddit.com/r/{}/about.json'.format(subreddit), headers = {'User-agent': 'reddit_crawler'}).json()
        start = int(r['data']['created_utc'])
    return start

In [7]:
def modify_meta(subreddit, last):
    meta = pd.read_csv('./data/metadata/data.csv', header=0)
    try:
        meta.loc[meta['subreddit'] == subreddit, 'last'] = last
        meta.to_csv('./data/metadata/data.csv', index=False)
    except:
        meta = meta.append({'subreddit':subreddit, 'last':last}, ignore_index=True)
        meta = meta[['subreddit', 'last']]
        meta.to_csv('./data/metadata/data.csv')

In [9]:
def get_reddit(subreddit, end_time):
    start_time = get_start_time(subreddit)

    arr = []
    base_url = 'https://api.pushshift.io/reddit/search/submission/?subreddit={subreddit}&after={start}&before={end}&sort=asc'
    get_all = False

    last_get_time = 0

    while not get_all:
        next_time = cal_time(start_time, 10)
        url = base_url.format(subreddit=subreddit, start=start_time, end=next_time)
        try:
            r = requests.get(url).json()
        except:
            r = {'data':[]}
            print(requests.get(url))

        if len(r['data']) == 0:
            if int(last_get_time) >= int(end_time):
                get_all = True
            else:
                start_time = next_time
                next_time = cal_time(start_time, 10)
        
        else:
            for submission in r['data']:
                # 날짜 분리
                timestamp = int(submission['created_utc'])
                d = datetime.fromtimestamp(timestamp)
                date = str(d.year)+'-'+str(d.month)+'-'+str(d.day)

                # 문자열에 대해 인코딩 처리
                sbr = encode_roman(submission['subreddit'])
                sbr_id = encode_roman(submission['subreddit_id'])
                uid = encode_roman(submission['id'])
                domain = encode_roman(submission['domain'])
                title = encode_roman(submission['title'])
                full_link = encode_roman(submission['full_link'])
                author = encode_roman(submission['author'])
                try:
                    if submission['selftext']:
                        selftext = encode_roman(submission['selftext'])
                    else:
                        selftext = ''
                except:
                    selftext = ''

                try:
                    if submission['thumbnail']:
                        thumbnail = encode_roman(submission['thumbnail'])
                    else:
                        thumbnail = ''
                except:
                    thumbnail = ''

                try:
                    if submission['media']['oembed']:
                        for e in submission['media']['oembed']:
                            media_provider = encode_roman(e['provider_name'])
                            media_thumbnail = encode_roman(e['thumbnail_url'])
                            media_title = encode_roman(e['title'])
                            media_description = encode_roman(e['description'])
                            media_url = encode_roman(e['url'])
                            break
                except:
                    media_provider = ''
                    media_thumbnail = ''
                    media_title = ''
                    media_description = ''
                    media_url = ''

                over_18 = str(submission['over_18'])

                # 데이터 저장
                if len(arr)>1 and d.month != arr[-1][4]:
                    get_all = True
                else:
                    tmp = [sbr, sbr_id, date, d.year, d.month, d.day, d.hour, uid, domain, title, full_link, 
                            author, submission['created_utc'], selftext, submission['num_comments'], 
                            submission['score'], over_18, thumbnail, media_provider,
                            media_thumbnail, media_title, media_description, media_url]
                    arr.append(tmp)

                # 마지막 날짜 기록용
                last_get_time = int(submission['created_utc'])

            start_time = last_get_time
            next_time = cal_time(start_time, 10)

    df_save = pd.DataFrame(arr)
    df_save.columns = ['sbr', 'sbr_id', 'date', 'year', 'month', 'hour', 'day', 'id', 
                        'domain', 'title', 'full_link', 'author', 'created', 'selftext', 
                        'num_comments', 'score', 'over_18', 'thumbnail', 'media_provider',
                        'media_thumbnail', 'media_title', 'media_description', 'media_url']


    dtm = datetime.fromtimestamp(last_get_time)
    yy = str(dtm.year)
    mm = str(dtm.month)

    if len(mm) < 2:
        mm = '0'+mm
        
    path = './data/scrapped/{dirname}/'.format(dirname = subreddit)
    save_file_name = path + '{yy}-{mm}-{last}.csv'.format(yy=yy, mm=mm, last=last_get_time)
    df_save.to_csv(save_file_name, encoding='cp949')
    modify_meta(subreddit, last_get_time)

In [ ]:
if __name__ == '__main__':
    os.chdir('./data/scrapped/')
    all_dirs = [os.path.abspath(name) for name in os.listdir(".") if os.path.isdir(name)]
    end_time = get_server_time()
    print(end_time)
    for dirc in all_dirs:
        subreddit = os.path.basename(os.path.normpath(dirc))
        print('crawling subreddit... ', subreddit)
        get_reddit(subreddit, end_time)

## API를 사용하는 크롤러 코드 예제

1) 공공데이터 API를 사용하는 방법, 일반적으로 API로 크롤링할 때의 방법을 잘 설명해 둠

[오픈 API를 통한 공공데이터 수집](https://medium.com/@whj2013123218/%EC%98%A4%ED%94%88-api%EB%A5%BC-%ED%86%B5%ED%95%9C-%EA%B3%B5%EA%B3%B5%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%88%98%EC%A7%91-e1dd0ad203b6)

2) 크롤링을 하면서 고려하게 되는 것들을 예시와 함께 설명해 둠

[네이버 API를 사용한 검색어 트렌드 크롤링 툴](https://brunch.co.kr/@sukhyun9673/13)

3) 그리고...
- 네이버, github, 트위터 등 크롤링 대상이 될 만한 데이터들이 많은 사이트에서는 대체로 API를 제공하고 있다.
- github에서는 완성된 크롤러 코드를 많이 볼 수 있다 (star 개수가 많은 레포지토리를 추천!)